# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /Users/tom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tom/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('messages_table', engine)
X = df.message
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [7]:
X.shape, Y.shape

((26028,), (26028, 36))

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    # normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower() ) 
    
    # splite text into words
    words = word_tokenize(text)
    
    # remove stop words 
    words = [w for w in words if w not in stopwords.words("english")]
    
    # lemmatization word
    words = [WordNetLemmatizer().lemmatize(w).strip() for w in words]
   
    #stemming word
    words = [PorterStemmer().stem(w) for w in words]  
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f8cf8492950>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
# predict on test data
Y_pred = pipeline.predict(X_test)

In [13]:
for i, col in enumerate(Y_test):
    print('Categories: {}'.format(col))
    print(classification_report(Y_test[col], Y_pred[:, i]))
print('Accuracy: {}'.format((Y_test.values == Y_pred).mean()))

Categories: related
              precision    recall  f1-score   support

           0       0.72      0.39      0.51      1583
           1       0.83      0.95      0.89      4924

    accuracy                           0.82      6507
   macro avg       0.78      0.67      0.70      6507
weighted avg       0.80      0.82      0.79      6507

Categories: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5397
           1       0.85      0.50      0.63      1110

    accuracy                           0.90      6507
   macro avg       0.88      0.74      0.78      6507
weighted avg       0.89      0.90      0.89      6507

Categories: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6476
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg    

/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6160
           1       0.81      0.11      0.19       347

    accuracy                           0.95      6507
   macro avg       0.88      0.55      0.58      6507
weighted avg       0.94      0.95      0.93      6507

Categories: electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6364
           1       1.00      0.02      0.04       143

    accuracy                           0.98      6507
   macro avg       0.99      0.51      0.52      6507
weighted avg       0.98      0.98      0.97      6507

Categories: tools
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6475
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.0

In [ ]:
pipeline.get_params()

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
cv_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

parameters = {  
      'clf__estimator__n_estimators': [20, 50], 
      'clf__estimator__min_samples_split': [4, 6]
} 

cv = GridSearchCV(cv_pipeline, param_grid=parameters, verbose = 4)

cv.fit(X_train, Y_train)

Y_pred = cv.predict(X_test)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20, score=0.242, total= 1.9min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.9min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20, score=0.249, total= 1.8min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20, score=0.264, total= 2.0min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20, score=0.253, total= 1.9min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=20, score=0.251, total= 1.8min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50, score=0.260, total= 2.9min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50, score=0.255, total= 2.8min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50, score=0.268, total= 2.7min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=50, score=0.262, total= 2.8min
[CV] cl

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 45.6min finished


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
for i, col in enumerate(Y_test):
    print('Categories: {}'.format(col))
    print(classification_report(Y_test[col], Y_pred[:, i]))
print('Accuracy: {}'.format((Y_test.values == Y_pred).mean()))

Categories: related
              precision    recall  f1-score   support

           0       0.73      0.39      0.51      1583
           1       0.83      0.95      0.89      4924

    accuracy                           0.82      6507
   macro avg       0.78      0.67      0.70      6507
weighted avg       0.80      0.82      0.79      6507

Categories: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5397
           1       0.85      0.52      0.65      1110

    accuracy                           0.90      6507
   macro avg       0.88      0.75      0.79      6507
weighted avg       0.90      0.90      0.89      6507

Categories: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6476
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg    

/Users/tom/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6475
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      6507

Categories: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6445
           1       0.00      0.00      0.00        62

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.98      0.99      0.99      6507

Categories: shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6479
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier

def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('svd', TruncatedSVD()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(MLPClassifier()))
    ])
    parameters = {
     'clf__estimator__early_stopping': [False, True],
     'clf__estimator__hidden_layer_sizes': [100, 200],
     'clf__estimator__learning_rate_init': [0.001, 0.01]
    }
    cv = GridSearchCV(pipeline, param_grid=parameters, verbose = 4)

    return cv

In [ ]:
mlp_model = build_model()
mlp_model.fit(X_train, Y_train)
Y_pred = mlp_model.predict(X_test)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.001 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.001, score=0.190, total= 2.5min
[CV] clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.001 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.001, score=0.181, total= 2.7min
[CV] clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.001 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.2min remaining:    0.0s


[CV]  clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.001, score=0.177, total= 2.8min
[CV] clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.001 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.0min remaining:    0.0s


[CV]  clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.001, score=0.188, total= 2.6min
[CV] clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.001 
[CV]  clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.001, score=0.185, total= 2.5min
[CV] clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.01 
[CV]  clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.01, score=0.190, total= 2.1min
[CV] clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.01 
[CV]  clf__estimator__early_stopping=False, clf__estimator__hidden_layer_sizes=100, clf__estimator__learning_rate_init=0.01, score=0.184, total= 2.3min
[CV] clf__estimat

In [ ]:
for i, col in enumerate(Y_test):
    print('Categories: {}'.format(col))
    print(classification_report(Y_test[col], Y_pred[:, i]))
print('Accuracy: {}'.format((Y_test.values == Y_pred).mean()))

### 9. Export your model as a pickle file

In [ ]:
import pickle

mlp_file_name = "pickle_mlp_model.pkl"  

with open(mlp_file_name, 'wb') as file:  
    pickle.dump(mlp_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# import libraries
import sys  # cmd input

import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'stopwords', 'wordnet'])

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier

def load_data(db_file_path):
    engine = create_engine('sqlite:///{}'.format(db_file_path))
    df = pd.read_sql_table('messages_table', engine)
    X = df.message
    y = df.drop(['id','message','original','genre'], axis=1).fillna(0)
    return X, y

def tokenize(text):
    # normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower() ) 
    # splite text into words
    words = word_tokenize(text)
    # remove stop words 
    words = [w for w in words if w not in stopwords.words("english")]
    # lemmatization word
    words = [WordNetLemmatizer().lemmatize(w).strip() for w in words]
    #stemming word
    words = [PorterStemmer().stem(w) for w in words]  
    return words

def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('svd', TruncatedSVD()),
        ('tfidf', TfidfTransformer()),
        ('clf', MLPClassifier())
    ])
    parameters = {
    'clf__early_stopping': (False, True),
    'clf__learning_rate': ('constant', 'invscaling', 'adaptive'),
    }
    model_pipeline = GridSearchCV(pipeline, param_grid=parameters, verbose = 4)
    return model_pipeline


def train(X, y, model):
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    # fit model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return model


def export_model(model):
    # Export model as a pickle file
    mlp_file_name = "pickle_mlp_model.pkl"  
    with open(mlp_file_name, 'wb') as file:  
        pickle.dump(model, file)


def run_pipeline(data_file):
    X, y = load_data(data_file)  # run ETL pipeline
    model = build_model()  # build model pipeline
    model = train(X, y, model)  # train model pipeline
    export_model(model)  # save model


if __name__ == '__main__':
    data_file = sys.argv[1]  # get filename of dataset
    run_pipeline(data_file)  # run data pipeline